# **Homework 9: Ιmplementing BB84 - Part I**
---

### **Description**
In this notebook, you will practice the python tools from this week's lab and work with the BB84 protocol.

<br>

### **Structure**
**Part 1**: [Learning the Tools](#p1)

**Part 2**: [Implementing BB84 in Python](#p2)
>
> **Part 2.1**: [The Protocol](#p2.1)
>
> **Part 2.2**: [Modifying the Protocol](#p2.2)

<br>

### **Learning Objectives**
By the end of this homework, you will:
* Recognize how to use three useful python tools: the choices function, dictionaries, and loops.

* Recognize how to implement the steps of BB84 between Alice and Bob using cirq.

<br>

### **Resources**
* [BB84 Cheat Sheet](https://docs.google.com/document/d/1FTBVWQsRVPvuP5e4lo3D62F0NOyfA1qIIPrDnocV6nc/edit)

<br>

**Before starting, run the code below to import all necessary functions and libraries.**

In [ ]:
!pip install cirq --quiet
import cirq

from random import choices

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.8/598.8 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 22.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

<a name="p2"></a>

---
## **Part 2: Implementing BB84 in Python**
---

The entire BB84 protocol between Alice and Bob is implemented below. In this part, you will interpret and adjust the code in order to understand it more deeply.

<a name="p2.1"></a>

---
### **Part 2.1: The Protocol**
---

The entire BB84 protocol between Alice and Bob is implemented below.

#### **The Setup**

In [ ]:
encode_gates = {0: cirq.I, 1: cirq.X}
basis_gates = {'Z': cirq.I, 'X': cirq.H}

num_bits = 5
qubits = cirq.NamedQubit.range(num_bits, prefix = 'q')

#### **Phase #1: Alice Sends**

In [ ]:
# Step #1
alice_key = choices([0, 1], k = num_bits)

print('Alice\'s initial key: ', alice_key)

# Step #2
alice_bases = choices(['Z', 'X'], k = num_bits)

print('\nAlice\'s randomly chosen bases: ', alice_bases)

# Step #3
alice_circuit = cirq.Circuit()

for bit in range(num_bits):

  encode_value = alice_key[bit]
  encode_gate = encode_gates[encode_value]

  basis_value = alice_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  alice_circuit.append(encode_gate(qubit))
  alice_circuit.append(basis_gate(qubit))

print('\nAlice\'s Phase 1 circuit:\n', alice_circuit)

Alice's initial key:  [1, 0, 1, 1, 0]

Alice's randomly chosen bases:  ['Z', 'Z', 'X', 'Z', 'X']

Alice's Phase 1 circuit:
 q0: ───X───I───

q1: ───I───I───

q2: ───X───H───

q3: ───X───I───

q4: ───I───H───


#### **Phase #2: Bob Receives**

In [ ]:
# Step #4
# No code required for this Step


# Step #5
bob_bases = choices(['Z', 'X'], k = num_bits)
print('Bob\'s randomly chosen bases: ', bob_bases)

bob_circuit = cirq.Circuit()

for bit in range(num_bits):

  basis_value = bob_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  bob_circuit.append(basis_gate(qubit))


# Step #6
bob_circuit.append(cirq.measure(qubits, key = 'bob key'))

print('\nBob\'s Phase 2 circuit:\n', bob_circuit)


# Step #7
bb84_circuit = alice_circuit + bob_circuit

sim = cirq.Simulator()
results = sim.run(bb84_circuit)
bob_key = results.measurements['bob key'][0]

print('\nBob\'s initial key: ', bob_key)

Bob's randomly chosen bases:  ['Z', 'Z', 'Z', 'X', 'X']

Bob's Phase 2 circuit:
 q0: ───I───M('bob key')───
           │
q1: ───I───M──────────────
           │
q2: ───I───M──────────────
           │
q3: ───H───M──────────────
           │
q4: ───H───M──────────────

Bob's initial key:  [1 0 1 0 0]


#### **Phase #3: Alice and Bob Compare**

In [ ]:
# Step #8
final_alice_key = []
final_bob_key = []

for bit in range(num_bits):

  if alice_bases[bit] == bob_bases[bit]:
    final_alice_key.append(alice_key[bit])
    final_bob_key.append(bob_key[bit])

print('\nAlice\'s key: ', final_alice_key)
print('Bob\'s key: ', final_bob_key)


# Step #9
if final_alice_key[0] == final_bob_key[0]:
  final_alice_key = final_alice_key[1:]
  final_bob_key = final_bob_key[1:]

  print('\n\nWe can use our keys!')
  print('Alice Key: ', final_alice_key)
  print('Bob Key: ', final_bob_key)

else:
  print('\n\nEve was listening, we need to use a different channel!')


Alice's key:  [1, 0, 0]
Bob's key:  [1, 0, 0]


We can use our keys!
Alice Key:  [0, 0]
Bob Key:  [0, 0]


<a name="p2.2"></a>

---
### **Part 2.2: Modifying the Protocol**
---

This part, you will run and modify the code in Part 2.1 in order to understand the protocol more deeply.

#### **Problem #2.1**

Run all the code above to perform the BB84 protocol as given. Answer the following questions in the multiple choice questions for this week's homework quiz:

* What is the smallest possible key this code can produce after performing all 9 steps?

* What is the largest possible key this code can produce after performing all 9 steps?

#### **Problem #2.2**

Now, modify the code above so that Alice creates an initial key that is 100 instead of 5 bits long. Then run the entire protocol again.

<br>

**Hint**: There is only one line of code that actually needs to change.

#### **Problem #2.3**

The BB84 protocol instructs Alice to encode the bits of her key into qubits as 0s and 1s first and then encode them into the Z or X bases.

<br>

**Modify the code in Phase #1 so that Alice encodes her qubits in the opposite order, in other words into the Z or X bases *before* encoding the bits of her key as 0s or 1s.** Then answer the associated multiple choice question in this week's homework quiz.

<br>

**NOTE**: Make sure you undo any changes from this Problem before moving on.

#### **Problem #2.4**

In Phases #1 - 2, we create two separate circuits: one for Alice's Steps and one for Bob's Steps.

<br>

This is not *strictly* necessary, so now **modify the code in Phase #2 so that Bob's steps are appended to `alice_circuit` instead of creating a new circuit.** Then answer the associated multiple choice question in this week's homework quiz.


<br>

**NOTE**: Make sure you undo any changes from this Problem before moving on. It may be easier to write your solution in a separate code cell rather than modify the code provided.

#### **Problem #2.5**

In Phase #3, Alice and Bob compare some of their key bits to ensure they actually agree. Currently, they just compare the very first bit of their keys.

<br>

**Modify the code in Phase #3 so that Alice and Bob compare the first two bits instead of just one.** Then answer the associated multiple choice question in this week's homework quiz.

# End of Notebook

---
© 2023 The Coding School, All rights reserved